In [3]:

import json


In [ ]:
import os
import json

# 修改成你的实验 log 文件夹
log_dir="/Users/racheleven/Desktop/graduate_courses/11-711/final_project_copy/results/civ_deepseek_spy_gpt_merged"

undercover_wins = 0
total_games = 0
undercover_game_ids = []

for fname in os.listdir(log_dir):
    if fname.startswith("game_log_") and fname.endswith(".json"):
        path = os.path.join(log_dir, fname)
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)

        total_games += 1
        game_id = data.get("metadata", {}).get("game_id")
        public_log = data.get("public_log", [])

        # ---- 卧底胜利标志 ----
        spy_alive_flag = False

        for entry in public_log:
            # phase 是 vote_result && round_num == 6 && "spy is still alive"
            if (
                entry.get("phase") == "vote_result"
                and entry.get("round_num") == 6
                and "spy is still alive" in entry.get("content", "")
            ):
                spy_alive_flag = True
                break

        if spy_alive_flag:
            undercover_wins += 1
            undercover_game_ids.append(game_id)

print("===== 卧底胜利统计结果 =====")
print(f"总游戏局数: {total_games}")
print(f"卧底胜利局数: {undercover_wins}")
print(f"卧底胜利比例: {undercover_wins / total_games:.2%}")
print("卧底胜利的 game_id:")
print(sorted(undercover_game_ids))


===== 卧底胜利统计结果 =====
总游戏局数: 50
卧底胜利局数: 0
卧底胜利比例: 0.00%
卧底胜利的 game_id:
[]


In [5]:
#每局游戏多少 round 结束
import os
import json

round_stats = {}  # game_id -> number of rounds

for fname in os.listdir(log_dir):
    if fname.startswith("game_log_") and fname.endswith(".json"):
        path = os.path.join(log_dir, fname)
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)

        game_id = data["metadata"]["game_id"]

        rounds = [entry["round_num"] for entry in data.get("public_log", []) 
                  if "round_num" in entry]

        max_round = max(rounds) if rounds else 0
        round_stats[game_id] = max_round

# 打印结果
for gid in sorted(round_stats):
    print(f"Game {gid}: {round_stats[gid]} rounds")

print("\n平均轮数:", sum(round_stats.values()) / len(round_stats))


Game 0: 1 rounds
Game 1: 1 rounds
Game 2: 2 rounds
Game 3: 2 rounds
Game 4: 1 rounds
Game 5: 3 rounds
Game 6: 1 rounds
Game 7: 1 rounds
Game 8: 1 rounds
Game 9: 2 rounds
Game 10: 1 rounds
Game 11: 1 rounds
Game 12: 1 rounds
Game 13: 1 rounds
Game 14: 1 rounds
Game 15: 1 rounds
Game 16: 5 rounds
Game 17: 1 rounds
Game 18: 1 rounds
Game 19: 1 rounds
Game 20: 1 rounds
Game 21: 2 rounds
Game 22: 1 rounds
Game 23: 3 rounds
Game 24: 1 rounds
Game 25: 3 rounds
Game 26: 1 rounds
Game 27: 2 rounds
Game 28: 3 rounds
Game 29: 6 rounds
Game 30: 1 rounds
Game 31: 1 rounds
Game 32: 1 rounds
Game 33: 1 rounds
Game 34: 1 rounds
Game 35: 1 rounds
Game 36: 1 rounds
Game 37: 2 rounds
Game 38: 1 rounds
Game 39: 1 rounds
Game 40: 1 rounds
Game 41: 1 rounds
Game 42: 1 rounds
Game 43: 1 rounds
Game 44: 2 rounds
Game 45: 2 rounds
Game 46: 3 rounds
Game 47: 1 rounds
Game 48: 1 rounds
Game 49: 1 rounds

平均轮数: 1.54


In [6]:
import json
import glob

# 结果存储
spy_games_count = {}            # {player_id: 该玩家多少局是卧底}
spy_self_discovery_count = {}   # {player_id: 在卧底局里多少局最后发现自己是卧底}

for file in glob.glob(f"{log_dir}/player_log_*.jsonl"):
    player_id = int(file.split("_")[-1].split(".")[0])

    spy_games_count[player_id] = 0
    spy_self_discovery_count[player_id] = 0

    with open(file, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)

            # ① 判断该局游戏玩家是否卧底
            is_spy = (data["metadata"]["role"] == "spy")
            if not is_spy:
                continue  # 平民局跳过

            spy_games_count[player_id] += 1

            # ② 该局游戏所有 log_info 中找最后一个 self_identity_guess
            last_guess_role = None

            for item in data["log_info"]:
                if item["phase"] == "self_identity_guess":
                    last_guess_role = item["guess_role"]  # "spy" / "civilian" / "unknown"

            # ③ 如果最后一次猜自己是 spy，则算作发现自己
            if last_guess_role == "spy":
                spy_self_discovery_count[player_id] += 1

# 打印结果
print("===== 每个玩家 卧底局 的 自我发现统计 =====")
for pid in sorted(spy_games_count.keys()):
    total_spy = spy_games_count[pid]
    found = spy_self_discovery_count[pid]
    rate = (found / total_spy * 100) if total_spy > 0 else 0.0

    print(f"Player {pid}:")
    print(f"  拿到卧底局数: {total_spy}")
    print(f"  其中最后发现自己是卧底的局数: {found}")
    print(f"  自我发现率: {rate:.1f}%")
    print()


===== 每个玩家 卧底局 的 自我发现统计 =====
Player 0:
  拿到卧底局数: 10
  其中最后发现自己是卧底的局数: 0
  自我发现率: 0.0%

Player 1:
  拿到卧底局数: 10
  其中最后发现自己是卧底的局数: 0
  自我发现率: 0.0%

Player 2:
  拿到卧底局数: 10
  其中最后发现自己是卧底的局数: 0
  自我发现率: 0.0%

Player 3:
  拿到卧底局数: 10
  其中最后发现自己是卧底的局数: 0
  自我发现率: 0.0%

Player 4:
  拿到卧底局数: 10
  其中最后发现自己是卧底的局数: 0
  自我发现率: 0.0%



In [8]:
import json
import glob

spy_discovery_games = {}    # {player_id: [game_id1, game_id2, ...]}

for file in glob.glob(f"{log_dir}/player_log_*.jsonl"):
    player_id = int(file.split("_")[-1].split(".")[0])
    spy_discovery_games[player_id] = []

    with open(file, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)

            game_id = data["metadata"]["game_id"]
            is_spy = (data["metadata"]["role"] == "spy")
            if not is_spy:
                continue  # 跳过平民局

            # 找该局的最后 self_identity_guess
            last_guess_role = None
            for item in data["log_info"]:
                if item["phase"] == "self_identity_guess":
                    last_guess_role = item["guess_role"]

            # 如果最后一次 guess_role 是 spy → 记录这个 game_id
            if last_guess_role == "spy":
                spy_discovery_games[player_id].append(game_id)

# 打印结果
print("===== 每个玩家 发现自己是卧底 的局 game_id =====")
for pid in sorted(spy_discovery_games.keys()):
    print(f"Player {pid}: {spy_discovery_games[pid]}")


===== 每个玩家 发现自己是卧底 的局 game_id =====
Player 0: []
Player 1: []
Player 2: []
Player 3: []
Player 4: []


In [ ]:
import json
import glob

# log_dir=
# 你已有的发现自己是卧底的局
# spy_discovery_games = {
#     0: [16, 19, 25, 40, 45, 49],
#     1: [3, 9, 10, 11, 24, 30, 31, 42, 47],
#     2: [12, 28, 29, 36, 37, 38, 39, 46, 48],
#     3: [1, 13, 15, 22, 23, 26, 34],
#     4: [2, 7, 8, 14, 33, 35, 41]
# }

# 结果：每个玩家在这些局存活了几轮
# survival_rounds = {pid: {} for pid in spy_discovery_games}

# for file in glob.glob(f"{log_dir}/player_log_*.jsonl"):
#     player_id = int(file.split("_")[-1].split(".")[0])

#     with open(file, "r", encoding="utf-8") as f:
#         for line in f:
#             data = json.loads(line)
#             game_id = data["metadata"]["game_id"]

#             # 只关心该玩家在“发现自己是卧底”的这些局
#             if game_id not in spy_discovery_games[player_id]:
#                 continue

#             # 找这个局中玩家的最大 round_num（= 存活轮数）
#             max_round = 0
#             for item in data["log_info"]:
#                 if "round_num" in item:
#                     max_round = max(max_round, item["round_num"])

#             survival_rounds[player_id][game_id] = max_round

# # 输出结果
# print("===== 玩家在发现自己是卧底的局中存活轮数 =====")
# for pid in sorted(survival_rounds.keys()):
#     print(f"Player {pid}:")
#     for gid, rounds in survival_rounds[pid].items():
#         print(f"  Game {gid}: survived {rounds} rounds")
#     print()


In [ ]:
# print 某个玩家某一局的具体输出

# file_path=

# target_game_id = 25

# with open(file_path, "r", encoding="utf-8") as f:
#     for line in f:
#         data = json.loads(line)
#         if data["metadata"]["game_id"] == target_game_id:
#             game15_log = data
#             break

# # 打印 log_info
# print(json.dumps(game15_log, indent=2, ensure_ascii=False))
